## Deploying the Trained Models

Refer to the previous notebooks for the training job and hyperparameter tuning job names.

In [ ]:
from sagemaker.tuner import HyperparameterTuner

# Replace with your hyperparameter tuning job name below
tuning_job_name="xgb-randsearch-20220412-18-10-58"

tuner = HyperparameterTuner.attach(tuning_job_name)

In [ ]:
tuner.best_training_job()

In [ ]:
predictor = tuner.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

## Send predictions

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import StringDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = StringDeserializer()

# Use Sample Data from 'test'
predictor.predict("28,69,1.0,0,0,1,0,0,1,0,0,1")

In [ ]:
import pandas as pd

test_data = pd.read_csv('test.csv', header=None)

# Test data without labels
test_data

for index, row, in test_data.iterrows():
    test_row = ','.join([str(i) for i in row.iloc[1:]])
    prediction = predictor.predict(test_row)
    print ("Predicted {} for Passenger {}".format(
        prediction,
        test_row))
    print(f"(True value was {row.iloc[0]})")
    test_data.loc[index, 'predicted'] = int(float(prediction))

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

print(f"accuracy {accuracy_score(test_data[0], test_data['predicted'])}")
conf_matrix = confusion_matrix(test_data[0], y_pred=test_data['predicted'])
print(conf_matrix)
sns.heatmap(conf_matrix, fmt='.3g', cmap='Blues', annot=True)

### Delete Endpoint

Not to incur charges

In [ ]:
import sagemaker

sagemaker.Session().delete_endpoint(endpoint_name=predictor.endpoint_name)

### Batch Prediction

Let's now run the inference as a Batch Transform Job

We read our test CSV file, and remove the first column, which is what we want to predict (survival)

In [ ]:
import pandas as pd
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

batchdf = pd.read_csv('test.csv',header=None)

batchdf = batchdf.drop(columns=0)

batchdf.to_csv("test_batch.csv",header=False,index=False)

input_batch = sagemaker_session.upload_data(path='test_batch.csv', key_prefix='titanic')

batchdf.head()

Now, let's create a transformer object that inherits the model and container used in the best training job we got from the hyperparameter tuning

In [ ]:
job_name = tuner.best_training_job()
estimator = sagemaker.estimator.Estimator.attach(job_name)

In [ ]:
# Normally you would not run batch manually, but trigger the batch prediction on an event 
# Some possible triggers would be: scheduled, a new file in S3

from sagemaker.transformer import Transformer

output_path='s3://{}/titanic'.format(bucket)

xgb_batch_transformer = estimator.transformer(
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=output_path
)

xgb_batch_transformer.base_transform_job_name='titanic-batch'

xgb_batch_transformer.transform(
    input_batch,
    content_type='text/csv',
    split_type='Line'
)

In [ ]:
!aws s3 ls {output_path}/

In [ ]:
!aws s3 cp {output_path}/test_batch.csv.out .